# Prompt Processing Historical View for the Past {{ params.days }} days

In [ ]:
# Times Square parameters
days = 7
instrument = "LSSTCam"
survey = "BLOCK-365"

In [ ]:
days = int(days)

In [ ]:
import astropy
from astropy.time import Time, TimeDelta
import matplotlib.pyplot as plt

%matplotlib inline
from lsst.daf.butler import Butler
from lsst.daf.butler.registry import MissingCollectionError

In [ ]:
butler = Butler("/repo/embargo", writeable=False)

In [ ]:
def print_summary(date, instrument=instrument):
    """
    Parameters
    ----------
    date : `astropy.time.Time`
    """
    date = date.to_value("iso", "date")
    dayobs = int(date.replace("-", ""))
    raws = butler.query_datasets(
        "raw",
        collections=f"{instrument}/raw/all",
        where="exposure.science_program=survey and "
        "instrument=instrument_name and day_obs=dayobs and detector<189 "
        bind={"dayobs": dayobs, "instrument_name": instrument, "survey": survey},
        limit=None, explain=False,
    )

    output_chain = f"{instrument}/prompt/output-{date}"
    if dayobs < 20240325:
        print(f"Collection chain started on 20240325; {dayobs} was too old.")
    try:
        refs = butler.query_datasets(
            "isr_log",
            collections=output_chain,
            where="instrument=instrument_name and day_obs=dayobs",
            bind={"dayobs": dayobs, "instrument_name": instrument},
            limit=None, explain=False,
        )
    except MissingCollectionError:
        refs = None
        data_count = 0
    else:
        data_count = len(refs)

    print(
        f"Found {len(raws):6d} raws on {dayobs} and {data_count:6d} outputs in {output_chain}"
    )
    return len(raws), data_count

In [ ]:
now = Time.now()
counts = {"raw": [], "output": []}
epoch = []
for x in range(days):
    date = now - TimeDelta(x * astropy.units.day)
    epoch.append(-x)

    raw, output = print_summary(date)
    counts["raw"].append(raw)
    counts["output"].append(output)

In [ ]:
fig, ax = plt.subplots()
for name, count in counts.items():
    p = ax.bar(epoch, count, label=name)
    ax.bar_label(p)
ax.legend()